## Create Dataset

In [6]:
#PEMBUATANDATASET

import cv2
import mediapipe as mp
import numpy as np
import pandas as pd

# Fungsi untuk menghitung landmark tangan
def extract_hand_landmarks(results):
    hand_landmarks_list = []
    for hand_land in results.multi_hand_landmarks:
        hand_landmarks = []
        for landmark in hand_land.landmark:
            hand_landmarks.extend([landmark.x, landmark.y])
        hand_landmarks_list.append(hand_landmarks)
    return hand_landmarks_list

# Inisialisasi video capture dari webcam
cap = cv2.VideoCapture(0)

# Inisialisasi mediapipe
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2)  # Ubah max_num_hands menjadi 2

# Meminta input kelas dan jumlah data
class_label = input("Masukkan label kelas (misal: Positif, Negatif): ")
num_samples = int(input("Masukkan jumlah data yang ingin direkam: "))

# Membuat list untuk menyimpan landmark tangan dan label kelas
hand_landmarks_list = []
class_labels = []

# Loop untuk merekam data landmark tangan
while len(hand_landmarks_list) < num_samples:
    success, frame = cap.read()
    if not success:
        break

    # Ubah frame menjadi RGB karena mediapipe menggunakan gambar RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Deteksi landmark tangan
    results = hands.process(image)

    # Jika landmark tangan terdeteksi, tambahkan ke dataset
    if results.multi_hand_landmarks:
        hand_landmarks = extract_hand_landmarks(results)
        hand_landmarks_list.extend(hand_landmarks)
        class_labels.extend([class_label] * len(hand_landmarks))

    # Tampilkan frame dengan landmark tangan
    if results.multi_hand_landmarks:
        for hand_land in results.multi_hand_landmarks:
            for idx, landmark in enumerate(hand_land.landmark):
                h, w, c = frame.shape
                cx, cy = int(landmark.x * w), int(landmark.y * h)
                cv2.circle(frame, (cx, cy), 5, (255, 0, 0), -1)
    cv2.imshow('Hand Landmarks', frame)

    # Keluar dari loop jika tombol 'q' ditekan
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Hentikan video capture dan tutup jendela
cap.release()
cv2.destroyAllWindows()

# Membuat DataFrame dari landmarks tangan dan label kelas setelah selesai merekam
new_df = pd.DataFrame(hand_landmarks_list, columns=[f'Landmark_{i}' for i in range(21*2)]) # 21 landmarks, masing-masing dengan x dan y
new_df.insert(0, 'Class', class_labels)  # Menyisipkan kolom 'Class' di awal DataFrame

# Baca DataFrame yang telah disimpan sebelumnya (jika ada)
try:
#     existing_df = pd.read_csv('Landmarks_kiri.csv') #untuk tangan kiri
#     existing_df = pd.read_csv('Landmarks_kanan.csv') #untuk tangan kanan
    existing_df = pd.read_csv('huruf.csv') #untuk tangan dua
    # Gabungkan data baru dengan data sebelumnya
    df = pd.concat([existing_df, new_df], ignore_index=True)
except FileNotFoundError:
    # Jika file CSV belum ada, langsung simpan data baru
    df = new_df

# Simpan DataFrame ke dalam file CSV
# df.to_csv('landmarks_kiri.csv', index=False) #kiri
# df.to_csv('landmarks_kanan.csv', index=False) #kanan
df.to_csv('huruf.csv', index=False) #dua
print("Data tangan berhasil disimpan ke .csv'.")


Masukkan label kelas (misal: Positif, Negatif): K
Masukkan jumlah data yang ingin direkam: 2000
Data tangan berhasil disimpan ke .csv'.


# Melatih Model

In [7]:
#TRAIN MODEL

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import joblib

# Baca data dari file CSV
# df = pd.read_csv('landmarks_kanan.csv') 
# df = pd.read_csv('landmarks_kiri.csv')
df = pd.read_csv('huruf.csv')

# Pisahkan fitur (X) dan label (y)
X = df.drop('Class', axis=1)
y = df['Class']

# Bagi dataset menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inisialisasi model SVM
model = SVC()

# Latih model menggunakan data latih
model.fit(X_train, y_train)

# Prediksi label untuk data uji
y_pred = model.predict(X_test)

# Hitung akurasi model
accuracy = accuracy_score(y_test, y_pred)
print(f'Akurasi model: {accuracy:.2f}')

# Simpan model yang telah dilatih ke dalam file .pkl
model_filename = 'huruf.pkl'
joblib.dump(model, model_filename)
print(f"Model disimpan sebagai '{model_filename}'")


Akurasi model: 1.00
Model disimpan sebagai 'huruf.pkl'


In [ ]:
#PENGUJIAN TANGAN KIRI/KANAN SAJA
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import joblib

# Fungsi untuk menghitung landmark tangan
def extract_hand_landmarks(results):
    hand_landmarks_list = []
    for hand_land in results.multi_hand_landmarks:
        hand_landmarks = []
        for landmark in hand_land.landmark:
            hand_landmarks.extend([landmark.x, landmark.y])
        hand_landmarks_list.append(hand_landmarks)
    return hand_landmarks_list

# Inisialisasi video capture dari webcam
cap = cv2.VideoCapture(0)

# Inisialisasi mediapipe
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)  # Ubah max_num_hands menjadi 2

# Muat model dari file .pkl
model = joblib.load('modelkanan.pkl')

# Loop untuk mendeteksi label kelas dari landmark tangan
while True:
    success, frame = cap.read()
    if not success:
        break

    # Ubah frame menjadi RGB karena mediapipe menggunakan gambar RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Deteksi landmark tangan
    results = hands.process(image)

    # Jika landmark tangan terdeteksi, lakukan prediksi
    if results.multi_hand_landmarks:
        hand_landmarks = extract_hand_landmarks(results)
        hand_landmarks_array = np.array(hand_landmarks)
        # Lakukan prediksi menggunakan model
        prediction = model.predict(hand_landmarks_array)
        class_label = prediction[0]
        
        # Tampilkan label kelas pada frame
        cv2.putText(frame, f"Hasil prediksi: {class_label}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        
        # Tampilkan frame dengan landmark tangan
        for hand_land in results.multi_hand_landmarks:
            for idx, landmark in enumerate(hand_land.landmark):
                h, w, c = frame.shape
                cx, cy = int(landmark.x * w), int(landmark.y * h)
                cv2.circle(frame, (cx, cy), 5, (255, 0, 0), -1)

    cv2.imshow('Hand Landmarks Detection', frame)

    # Keluar dari loop jika tombol 'q' ditekan
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Hentikan video capture dan tutup jendela
cap.release()
cv2.destroyAllWindows()


In [ ]:
#3 model
import cv2
import mediapipe as mp
import joblib
import numpy as np

# Fungsi untuk menghitung landmark tangan
def extract_hand_landmarks(results):
    hand_landmarks_list = []
    for hand_land in results.multi_hand_landmarks:
        hand_landmarks = []
        for landmark in hand_land.landmark:
            hand_landmarks.extend([landmark.x, landmark.y])
        hand_landmarks_list.append(hand_landmarks)
    return hand_landmarks_list

# Inisialisasi video capture dari webcam
cap = cv2.VideoCapture(0)
# Kurangi resolusi frame
cap.set(3, 640)  # Lebar frame: 640 piksel
cap.set(4, 480)  # Tinggi frame: 480 piksel

# Inisialisasi mediapipe
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)  # Ubah max_num_hands menjadi 2

# Muat model untuk tangan kanan dari file .pkl
model_right_hand = joblib.load('modelkiri.pkl')

# Muat model untuk tangan kiri dari file .pkl
model_left_hand = joblib.load('modelkanan.pkl')

# Muat model untuk dua tangan dari file .pkl
model_two_hands = joblib.load('modelduatangan.pkl')

# Variabel untuk menyimpan kelas yang terdeteksi sebelumnya
last_class_label = None

# Loop untuk mendeteksi label kelas dari landmark tangan
while True:
    success, frame = cap.read()
    if not success:
        break

    # Ubah frame menjadi RGB karena mediapipe menggunakan gambar RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Deteksi landmark tangan
    results = hands.process(image)

    # Jika landmark tangan terdeteksi
    if results.multi_hand_landmarks:
        num_hands = len(results.multi_hand_landmarks)
        if num_hands == 1:
            # Deteksi untuk satu tangan
            hand_landmarks = extract_hand_landmarks(results)
            hand_landmarks_array = np.array(hand_landmarks)

            # Reshape hand_landmarks_array to 2D array
            hand_landmarks_array = hand_landmarks_array.reshape(-1, 42)  # Reshape to (num_samples, 42)

            # Lakukan prediksi menggunakan model_right_hand atau model_left_hand
            # Berdasarkan posisi tangan (dapat menggunakan landmark tangan untuk membedakannya)
            hand_landmarks_right = extract_hand_landmarks(results)[0]
            if hand_landmarks_right[0] > hand_landmarks_right[17]:
                prediction = model_right_hand.predict(hand_landmarks_array)
            else:
                prediction = model_left_hand.predict(hand_landmarks_array)
            class_label = prediction[0]
        elif num_hands == 2:
            # Deteksi untuk dua tangan
            hand_landmarks_left = extract_hand_landmarks(results)[0]  # Landmarks for left hand
            hand_landmarks_right = extract_hand_landmarks(results)[1]  # Landmarks for right hand

            # Lakukan prediksi menggunakan model_two_hands
            hand_landmarks_array = np.array([hand_landmarks_left, hand_landmarks_right])
            # Reshape hand_landmarks_array to 2D array
            hand_landmarks_array = hand_landmarks_array.reshape(-1, 42)  # Reshape to (num_samples, 42)

            prediction = model_two_hands.predict(hand_landmarks_array)
            class_label = prediction[0]
        else:
            # Kasus ketika lebih dari dua tangan terdeteksi, tidak akan diproses dalam contoh ini.
            continue

        # Tampilkan label kelas pada frame
        cv2.putText(frame, f"{class_label}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        # Simpan kelas yang terdeteksi saat ini untuk perbandingan berikutnya
        last_class_label = class_label

    cv2.imshow('Hand Landmarks Detection', frame)

    # Keluar dari loop jika tombol 'q' ditekan
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Hentikan video capture dan tutup jendela
cap.release()
cv2.destroyAllWindows()

In [1]:
#OUTPUT SUARA

#3 model
import cv2
import mediapipe as mp
import joblib
import numpy as np
import pygame

# Fungsi untuk menghitung landmark tangan
def extract_hand_landmarks(results):
    hand_landmarks_list = []
    for hand_land in results.multi_hand_landmarks:
        hand_landmarks = []
        for landmark in hand_land.landmark:
            hand_landmarks.extend([landmark.x, landmark.y])
        hand_landmarks_list.append(hand_landmarks)
    return hand_landmarks_list


# Initialize Pygame
pygame.init()

# Inisialisasi pygame.mixer.Channel untuk mengontrol output suara
sound_channel = pygame.mixer.Channel(0)

# Flag untuk menandai apakah suara sudah dimainkan atau belum
sound_played = False

# Variabel untuk menyimpan kelas yang terdeteksi sebelumnya
last_class_label = None

# # Load the sound files
# sound_folder = "sound"
# sound_files = {}
# for filename in os.listdir(sound_folder):
#     if filename.endswith(".wav"):
#         class_label = os.path.splitext(filename)[0]
#         sound_files[class_label] = pygame.mixer.Sound(os.path.join(sound_folder, filename))


# Inisialisasi video capture dari webcam
cap = cv2.VideoCapture(0)
# Kurangi resolusi frame
cap.set(3, 640)  # Lebar frame: 640 piksel
cap.set(4, 480)  # Tinggi frame: 480 piksel

# Inisialisasi mediapipe
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)  # Ubah max_num_hands menjadi 2

# Muat model untuk tangan kanan dari file .pkl
model_right_hand = joblib.load('modelkiri.pkl')

# Muat model untuk tangan kiri dari file .pkl
model_left_hand = joblib.load('modelkanan.pkl')

# Muat model untuk dua tangan dari file .pkl
model_two_hands = joblib.load('modelduatangan.pkl')

# Variabel untuk menyimpan kelas yang terdeteksi sebelumnya
last_class_label = None

# Loop untuk mendeteksi label kelas dari landmark tangan
while True:
    success, frame = cap.read()
    if not success:
        break

    # Ubah frame menjadi RGB karena mediapipe menggunakan gambar RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Deteksi landmark tangan
    results = hands.process(image)

    # Jika landmark tangan terdeteksi
    if results.multi_hand_landmarks:
        num_hands = len(results.multi_hand_landmarks)
        if num_hands == 1:
            # Deteksi untuk satu tangan
            hand_landmarks = extract_hand_landmarks(results)
            hand_landmarks_array = np.array(hand_landmarks)

            # Reshape hand_landmarks_array to 2D array
            hand_landmarks_array = hand_landmarks_array.reshape(-1, 42)  # Reshape to (num_samples, 42)

            # Lakukan prediksi menggunakan model_right_hand atau model_left_hand
            # Berdasarkan posisi tangan (dapat menggunakan landmark tangan untuk membedakannya)
            hand_landmarks_right = extract_hand_landmarks(results)[0]
            if hand_landmarks_right[0] > hand_landmarks_right[17]:
                prediction = model_right_hand.predict(hand_landmarks_array)
            else:
                prediction = model_left_hand.predict(hand_landmarks_array)
            class_label = prediction[0]
        elif num_hands == 2:
            # Deteksi untuk dua tangan
            hand_landmarks_left = extract_hand_landmarks(results)[0]  # Landmarks for left hand
            hand_landmarks_right = extract_hand_landmarks(results)[1]  # Landmarks for right hand

            # Lakukan prediksi menggunakan model_two_hands
            hand_landmarks_array = np.array([hand_landmarks_left, hand_landmarks_right])
            # Reshape hand_landmarks_array to 2D array
            hand_landmarks_array = hand_landmarks_array.reshape(-1, 42)  # Reshape to (num_samples, 42)

            prediction = model_two_hands.predict(hand_landmarks_array)
            class_label = prediction[0]
        else:
            # Kasus ketika lebih dari dua tangan terdeteksi, tidak akan diproses dalam contoh ini.
            continue

        # Tampilkan label kelas pada frame
        cv2.putText(frame, f"{class_label}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Output suara menggunakan pygame hanya jika belum dimainkan sebelumnya
        sound_file = f'{class_label}.mp3'  # Assuming you have mp3 files for each class label
        if last_class_label != class_label and os.path.exists(sound_file):
            sound_channel.play(pygame.mixer.Sound(sound_file))
            sound_played = True
        else:
            sound_played = False

        # Simpan kelas yang terdeteksi saat ini untuk perbandingan berikutnya
        last_class_label = class_label


    cv2.imshow('Hand Landmarks Detection', frame)

    # Keluar dari loop jika tombol 'q' ditekan
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Hentikan video capture dan tutup jendela
cap.release()
cv2.destroyAllWindows()

pygame 2.3.0 (SDL 2.24.2, Python 3.8.2)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [1]:
#OUTPUT SUARA fix banget tinggal gabungin face tracking

#3 model
import os
import cv2
import mediapipe as mp
import joblib
import numpy as np

# Fungsi untuk menghitung landmark tangan
def extract_hand_landmarks(results):
    hand_landmarks_list = []
    for hand_land in results.multi_hand_landmarks:
        hand_landmarks = []
        for landmark in hand_land.landmark:
            hand_landmarks.extend([landmark.x, landmark.y])
        hand_landmarks_list.append(hand_landmarks)
    return hand_landmarks_list


# Initialize Pygame
pygame.init()

# Inisialisasi pygame.mixer.Channel untuk mengontrol output suara
sound_channel = pygame.mixer.Channel(0)

# Flag untuk menandai apakah suara sudah dimainkan atau belum
sound_played = False

# Variabel untuk menyimpan kelas yang terdeteksi sebelumnya
last_class_label = None


# Inisialisasi video capture dari webcam
cap = cv2.VideoCapture(0)
# Kurangi resolusi frame
cap.set(3, 640)  # Lebar frame: 640 piksel
cap.set(4, 480)  # Tinggi frame: 480 piksel

# Inisialisasi mediapipe
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)  # Ubah max_num_hands menjadi 2

# Muat model untuk tangan kanan dari file .pkl
model_right_hand = joblib.load('modelkiri.pkl')

# Muat model untuk tangan kiri dari file .pkl
model_left_hand = joblib.load('modelkanan.pkl')

# Muat model untuk dua tangan dari file .pkl
model_two_hands = joblib.load('modelduatangan.pkl')

# Variabel untuk menyimpan kelas yang terdeteksi sebelumnya
last_class_label = None

# Before the while loop, load the sound files
sound_folder = "sound"
sound_files = {}
for filename in os.listdir(sound_folder):
    if filename.endswith(".mp3"):
        class_label = os.path.splitext(filename)[0]
        sound_files[class_label] = os.path.join(sound_folder, filename)

# Loop untuk mendeteksi label kelas dari landmark tangan
while True:
    success, frame = cap.read()
    if not success:
        break

    # Ubah frame menjadi RGB karena mediapipe menggunakan gambar RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Deteksi landmark tangan
    results = hands.process(image)

    # Jika landmark tangan terdeteksi
    if results.multi_hand_landmarks:
        num_hands = len(results.multi_hand_landmarks)
        if num_hands == 1:
            # Deteksi untuk satu tangan
            hand_landmarks = extract_hand_landmarks(results)
            hand_landmarks_array = np.array(hand_landmarks)

            # Reshape hand_landmarks_array to 2D array
            hand_landmarks_array = hand_landmarks_array.reshape(-1, 42)  # Reshape to (num_samples, 42)

            # Lakukan prediksi menggunakan model_right_hand atau model_left_hand
            # Berdasarkan posisi tangan (dapat menggunakan landmark tangan untuk membedakannya)
            hand_landmarks_right = extract_hand_landmarks(results)[0]
            if hand_landmarks_right[0] > hand_landmarks_right[17]:
                prediction = model_right_hand.predict(hand_landmarks_array)
            else:
                prediction = model_left_hand.predict(hand_landmarks_array)
            class_label = prediction[0]
        elif num_hands == 2:
            # Deteksi untuk dua tangan
            hand_landmarks_left = extract_hand_landmarks(results)[0]  # Landmarks for left hand
            hand_landmarks_right = extract_hand_landmarks(results)[1]  # Landmarks for right hand

            # Lakukan prediksi menggunakan model_two_hands
            hand_landmarks_array = np.array([hand_landmarks_left, hand_landmarks_right])
            # Reshape hand_landmarks_array to 2D array
            hand_landmarks_array = hand_landmarks_array.reshape(-1, 42)  # Reshape to (num_samples, 42)

            prediction = model_two_hands.predict(hand_landmarks_array)
            class_label = prediction[0]
        else:
            # Kasus ketika lebih dari dua tangan terdeteksi, tidak akan diproses dalam contoh ini.
            continue

        # Tampilkan label kelas pada frame
        cv2.putText(frame, f"{class_label}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        
        # Output suara menggunakan pygame hanya jika belum dimainkan sebelumnya
        sound_file = f'{class_label}.mp3'  # Assuming you have mp3 files for each class label
         # Output suara menggunakan pygame hanya jika belum dimainkan sebelumnya
        if last_class_label != class_label and class_label in sound_files:
            sound_file = sound_files[class_label]
            sound_channel.play(pygame.mixer.Sound(sound_file))
            sound_played = True
        else:
            sound_played = False
        # Simpan kelas yang terdeteksi saat ini untuk perbandingan berikutnya
        last_class_label = class_label


    cv2.imshow('Hand Landmarks Detection', frame)

    # Keluar dari loop jika tombol 'q' ditekan
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Hentikan video capture dan tutup jendela
cap.release()
cv2.destroyAllWindows()

NameError: name 'pygame' is not defined

In [9]:
#OUTPUT SUARA fix banget tinggal gabungin face tracking 2

#3 model
import os
import cv2
import mediapipe as mp
import joblib
import numpy as np
import pygame



# Fungsi untuk menghitung landmark tangan
def extract_hand_landmarks(results):
    hand_landmarks_list = []
    for hand_land in results.multi_hand_landmarks:
        hand_landmarks = []
        for landmark in hand_land.landmark:
            hand_landmarks.extend([landmark.x, landmark.y])
        hand_landmarks_list.append(hand_landmarks)
    return hand_landmarks_list


# Initialize Pygame
pygame.init()

# Inisialisasi pygame.mixer.Channel untuk mengontrol output suara
sound_channel = pygame.mixer.Channel(0)

# Flag untuk menandai apakah suara sudah dimainkan atau belum
sound_played = False

# Variabel untuk menyimpan kelas yang terdeteksi sebelumnya
last_class_label = None


# Inisialisasi video capture dari webcam
cap = cv2.VideoCapture(0)
# Kurangi resolusi frame
cap.set(3, 640)  # Lebar frame: 640 piksel
cap.set(4, 480)  # Tinggi frame: 480 piksel

# Inisialisasi mediapipe
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)  # Ubah max_num_hands menjadi 2

# Muat model untuk tangan kanan dari file .pkl
model_right_hand = joblib.load('modelkiri.pkl')

# Muat model untuk tangan kiri dari file .pkl
model_left_hand = joblib.load('huruf.pkl')

# Muat model untuk dua tangan dari file .pkl
model_two_hands = joblib.load('modelduatangan.pkl')

# Variabel untuk menyimpan kelas yang terdeteksi sebelumnya
last_class_label = None

# Before the while loop, load the sound files
sound_folder = "sound"
sound_files = {}
for filename in os.listdir(sound_folder):
    if filename.endswith(".mp3"):
        class_label = os.path.splitext(filename)[0]
        sound_files[class_label] = os.path.join(sound_folder, filename)

# Loop untuk mendeteksi label kelas dari landmark tangan
while True:
    success, frame = cap.read()
    if not success:
        break

    # Ubah frame menjadi RGB karena mediapipe menggunakan gambar RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Deteksi landmark tangan
    results = hands.process(image)

    # Jika landmark tangan terdeteksi
    if results.multi_hand_landmarks:
        num_hands = len(results.multi_hand_landmarks)
        if num_hands == 1:
            # Deteksi untuk satu tangan
            hand_landmarks = extract_hand_landmarks(results)
            hand_landmarks_array = np.array(hand_landmarks)

            # Reshape hand_landmarks_array to 2D array
            hand_landmarks_array = hand_landmarks_array.reshape(-1, 42)  # Reshape to (num_samples, 42)

            # Lakukan prediksi menggunakan model_right_hand atau model_left_hand
            # Berdasarkan posisi tangan (dapat menggunakan landmark tangan untuk membedakannya)
            hand_landmarks_right = extract_hand_landmarks(results)[0]
            if hand_landmarks_right[0] > hand_landmarks_right[17]:
                prediction = model_right_hand.predict(hand_landmarks_array)
            else:
                prediction = model_left_hand.predict(hand_landmarks_array)
            class_label = prediction[0]
        elif num_hands == 2:
            # Deteksi untuk dua tangan
            hand_landmarks_left = extract_hand_landmarks(results)[0]  # Landmarks for left hand
            hand_landmarks_right = extract_hand_landmarks(results)[1]  # Landmarks for right hand

            # Lakukan prediksi menggunakan model_two_hands
            hand_landmarks_array = np.array([hand_landmarks_left, hand_landmarks_right])
            # Reshape hand_landmarks_array to 2D array
            hand_landmarks_array = hand_landmarks_array.reshape(-1, 42)  # Reshape to (num_samples, 42)

            prediction = model_two_hands.predict(hand_landmarks_array)
            class_label = prediction[0]
        else:
            # Kasus ketika lebih dari dua tangan terdeteksi, tidak akan diproses dalam contoh ini.
            continue

        # Tampilkan label kelas pada frame
        cv2.putText(frame, f"{class_label}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        
        # Output suara menggunakan pygame hanya jika belum dimainkan sebelumnya
        sound_file = f'{class_label}.mp3'  # Assuming you have mp3 files for each class label
         # Output suara menggunakan pygame hanya jika belum dimainkan sebelumnya
        if last_class_label != class_label and class_label in sound_files:
            sound_file = sound_files[class_label]
            sound_channel.play(pygame.mixer.Sound(sound_file))
            sound_played = True
        else:
            sound_played = False
        # Simpan kelas yang terdeteksi saat ini untuk perbandingan berikutnya
        last_class_label = class_label


    cv2.imshow('Hand Landmarks Detection', frame)

    # Keluar dari loop jika tombol 'q' ditekan
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Hentikan video capture dan tutup jendela
cap.release()
cv2.destroyAllWindows()

C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\skle

C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\skle

C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\skle

C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\skle

C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\skle

...

C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\skle

C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\skle

C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\skle

C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\skle

C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\skle

In [ ]:
#OUTPUT SUARA fix banget tinggal gabungin face tracking 2

import pygame
import os
import cv2
import mediapipe as mp
import joblib
import numpy as np
import time
import serial



# Fungsi untuk menghitung landmark tangan
def extract_hand_landmarks(results):
    hand_landmarks_list = []
    for hand_land in results.multi_hand_landmarks:
        hand_landmarks = []
        for landmark in hand_land.landmark:
            hand_landmarks.extend([landmark.x, landmark.y])
        hand_landmarks_list.append(hand_landmarks)
    return hand_landmarks_list


# Initialize Pygame
pygame.init()

# Inisialisasi pygame.mixer.Channel untuk mengontrol output suara
sound_channel = pygame.mixer.Channel(0)

# Flag untuk menandai apakah suara sudah dimainkan atau belum
sound_played = False

# Variabel untuk menyimpan kelas yang terdeteksi sebelumnya
last_class_label = None


# Inisialisasi video capture dari webcam
cap = cv2.VideoCapture(0)
# Kurangi resolusi frame
cap.set(3, 640)  # Lebar frame: 640 piksel
cap.set(4, 480)  # Tinggi frame: 480 piksel

# Inisialisasi mediapipe
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)  # Ubah max_num_hands menjadi 2

# Muat model untuk tangan kanan dari file .pkl
model_right_hand = joblib.load('modelkiri.pkl')

# Muat model untuk tangan kiri dari file .pkl
model_left_hand = joblib.load('modelkanan.pkl')

# Muat model untuk dua tangan dari file .pkl
model_two_hands = joblib.load('modelduatangan.pkl')

# Variabel untuk menyimpan kelas yang terdeteksi sebelumnya
last_class_label = None

# Before the while loop, load the sound files
sound_folder = "sound"
sound_files = {}
for filename in os.listdir(sound_folder):
    if filename.endswith(".mp3"):
        class_label = os.path.splitext(filename)[0]
        sound_files[class_label] = os.path.join(sound_folder, filename)

# Loop untuk mendeteksi label kelas dari landmark tangan
while True:
    success, frame = cap.read()
    if not success:
        break

    # Ubah frame menjadi RGB karena mediapipe menggunakan gambar RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Deteksi landmark tangan
    results = hands.process(image)

    # Jika landmark tangan terdeteksi
    if results.multi_hand_landmarks:
        num_hands = len(results.multi_hand_landmarks)
        if num_hands == 1:
            # Deteksi untuk satu tangan
            hand_landmarks = extract_hand_landmarks(results)
            hand_landmarks_array = np.array(hand_landmarks)

            # Reshape hand_landmarks_array to 2D array
            hand_landmarks_array = hand_landmarks_array.reshape(-1, 42)  # Reshape to (num_samples, 42)

            # Lakukan prediksi menggunakan model_right_hand atau model_left_hand
            # Berdasarkan posisi tangan (dapat menggunakan landmark tangan untuk membedakannya)
            hand_landmarks_right = extract_hand_landmarks(results)[0]
            if hand_landmarks_right[0] > hand_landmarks_right[17]:
                prediction = model_right_hand.predict(hand_landmarks_array)
            else:
                prediction = model_left_hand.predict(hand_landmarks_array)
            class_label = prediction[0]
        elif num_hands == 2:
            # Deteksi untuk dua tangan
            hand_landmarks_left = extract_hand_landmarks(results)[0]  # Landmarks for left hand
            hand_landmarks_right = extract_hand_landmarks(results)[1]  # Landmarks for right hand

            # Lakukan prediksi menggunakan model_two_hands
            hand_landmarks_array = np.array([hand_landmarks_left, hand_landmarks_right])
            # Reshape hand_landmarks_array to 2D array
            hand_landmarks_array = hand_landmarks_array.reshape(-1, 42)  # Reshape to (num_samples, 42)

            prediction = model_two_hands.predict(hand_landmarks_array)
            class_label = prediction[0]
        else:
            # Kasus ketika lebih dari dua tangan terdeteksi, tidak akan diproses dalam contoh ini.
            continue

        # Tampilkan label kelas pada frame
        cv2.putText(frame, f"{class_label}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        
        # Output suara menggunakan pygame hanya jika belum dimainkan sebelumnya
        sound_file = f'{class_label}.mp3'  # Assuming you have mp3 files for each class label
         # Output suara menggunakan pygame hanya jika belum dimainkan sebelumnya
        if last_class_label != class_label and class_label in sound_files:
            sound_file = sound_files[class_label]
            sound_channel.play(pygame.mixer.Sound(sound_file))
            sound_played = True
        else:
            sound_played = False
        # Simpan kelas yang terdeteksi saat ini untuk perbandingan berikutnya
        last_class_label = class_label


    cv2.imshow('Hand Landmarks Detection', frame)

    # Keluar dari loop jika tombol 'q' ditekan
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Hentikan video capture dan tutup jendela
cap.release()
cv2.destroyAllWindows()
mp_face_detection = mp.solutions.face_detection

# Konfigurasi Serial Port
ser = serial.Serial('COM16', 9600)  # Ganti dengan port dan baudrate yang sesuai
time.sleep(2)  # Beri waktu beberapa detik untuk memulai komunikasi

# Inisialisasi MediaPipe Face Detection
face_detection = mp_face_detection.FaceDetection()

cap = cv2.VideoCapture(1)  # Ganti dengan nomor video device yang sesuai

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)  # Mirror the image

    # Ubah gambar menjadi RGB untuk MediaPipe
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Deteksi wajah menggunakan MediaPipe
    results = face_detection.process(frame_rgb)

    if results.detections:
        for detection in results.detections:
            # Ambil koordinat kotak pembatas wajah
            bbox = detection.location_data.relative_bounding_box
            x, y, w, h = int(bbox.xmin * frame.shape[1]), int(bbox.ymin * frame.shape[0]), \
                         int(bbox.width * frame.shape[1]), int(bbox.height * frame.shape[0])

            # Kirim koordinat tengah wajah ke Arduino
            string = 'X{0:d}Y{1:d}'.format(x + w//2, y + h//2)
            print(string)
            ser.write(string.encode('utf-8'))
            time.sleep(0.2)

            # Gambar kotak pembatas wajah
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 3)
            # Plot titik tengah wajah
            cv2.circle(frame, (x + w//2, y + h//2), 2, (0, 255, 0), 2)

    # Gambar area persegi di tengah layar
    cv2.rectangle(frame, (640//2-30, 480//2-30), (640//2+30, 480//2+30), (255, 255, 255), 3)

    cv2.imshow('Face Detection', frame)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
ser.close()

In [1]:
#kode final

import os
import cv2
import mediapipe as mp
import joblib
import numpy as np
import pygame
import serial
import time
import threading


cap = cv2.VideoCapture(1)
# Inisialisasi Pygame
pygame.init()
mp_face_detection = mp.solutions.face_detection

# Inisialisasi Pygame.mixer.Channel untuk mengontrol output suara
sound_channel = pygame.mixer.Channel(0)
# Initialize video capture dari webcam


#   # Konfigurasi Serial Port
ser = serial.Serial('COM16', 9600)  # Ganti dengan port dan baudrate yang sesuai
time.sleep(2)  # Beri waktu beberapa detik untuk memulai komunikasi

# Variabel untuk menyimpan kelas yang terdeteksi sebelumnya
last_class_label = None

# Fungsi untuk menghitung landmark tangan
def extract_hand_landmarks(results):
    hand_landmarks_list = []
    for hand_land in results.multi_hand_landmarks:
        hand_landmarks = []
        for landmark in hand_land.landmark:
            hand_landmarks.extend([landmark.x, landmark.y])
        hand_landmarks_list.append(hand_landmarks)
    return hand_landmarks_list

def hand_detection():
    global last_class_label  # Declare last_class_label as a global variable
    
    

#     # Kurangi resolusi frame
#     cap.set(3, 640)  # Lebar frame: 640 piksel
#     cap.set(4, 480)  # Tinggi frame: 480 piksel

    # Inisialisasi mediapipe untuk tangan
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(
        static_image_mode=False,
        max_num_hands=2,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5)  # Ubah max_num_hands menjadi 2

    # Muat model untuk tangan kanan dari file .pkl
    model_right_hand = joblib.load('modelkiri.pkl')

    # Muat model untuk tangan kiri dari file .pkl
    model_left_hand = joblib.load('huruf.pkl')

    # Muat model untuk dua tangan dari file .pkl
    model_two_hands = joblib.load('modelduatangan.pkl')

    # Before the while loop, load the sound files
    sound_folder = "sound"
    sound_files = {}
    for filename in os.listdir(sound_folder):
        if filename.endswith(".mp3"):
            class_label = os.path.splitext(filename)[0]
            sound_files[class_label] = os.path.join(sound_folder, filename)

    while cap.isOpened():
        # Capture frame
        success, frame = cap.read()
        if not success:
            break

        # Clone frame untuk tangan
        frame_hand = frame.copy()

        # Ubah frame menjadi RGB karena mediapipe menggunakan gambar RGB
        image_hand = cv2.cvtColor(frame_hand, cv2.COLOR_BGR2RGB)

        # Deteksi landmark tangan
        results = hands.process(image_hand)

        # Jika landmark tangan terdeteksi
        if results.multi_hand_landmarks:
            num_hands = len(results.multi_hand_landmarks)
            if num_hands == 1:
                # Deteksi untuk satu tangan
                hand_landmarks = extract_hand_landmarks(results)
                hand_landmarks_array = np.array(hand_landmarks)

                # Reshape hand_landmarks_array to 2D array
                hand_landmarks_array = hand_landmarks_array.reshape(-1, 42)  # Reshape to (num_samples, 42)

                # Lakukan prediksi menggunakan model_right_hand atau model_left_hand
                # Berdasarkan posisi tangan (dapat menggunakan landmark tangan untuk membedakannya)
                hand_landmarks_right = extract_hand_landmarks(results)[0]
                if hand_landmarks_right[0] > hand_landmarks_right[17]:
                    prediction = model_right_hand.predict(hand_landmarks_array)
                else:
                    prediction = model_left_hand.predict(hand_landmarks_array)
                class_label = prediction[0]
            elif num_hands == 2:
                # Deteksi untuk dua tangan
                hand_landmarks_left = extract_hand_landmarks(results)[0]  # Landmarks for left hand
                hand_landmarks_right = extract_hand_landmarks(results)[1]  # Landmarks for right hand

                # Lakukan prediksi menggunakan model_two_hands
                hand_landmarks_array = np.array([hand_landmarks_left, hand_landmarks_right])
                # Reshape hand_landmarks_array to 2D array
                hand_landmarks_array = hand_landmarks_array.reshape(-1, 42)  # Reshape to (num_samples, 42)

                prediction = model_two_hands.predict(hand_landmarks_array)
                class_label = prediction[0]
            else:
                # Kasus ketika lebih dari dua tangan terdeteksi, tidak akan diproses dalam contoh ini.
                continue

            # Tampilkan label kelas pada frame tangan
            cv2.putText(frame_hand, f"{class_label}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            # Output suara menggunakan pygame hanya jika belum dimainkan sebelumnya
            sound_file = f'{class_label}.mp3'  # Assuming you have mp3 files for each class label
            if last_class_label != class_label and class_label in sound_files:
                sound_file = sound_files[class_label]
                sound_channel.play(pygame.mixer.Sound(sound_file))
            last_class_label = class_label

        # Tampilkan frame tangan di layar
        cv2.imshow('Hand Detection', frame_hand)

        # Keluar dari loop jika tombol 'q' ditekan
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

def face_detection_worker():
    # Deteksi wajah menggunakan MediaPipe
    with mp_face_detection.FaceDetection() as face_detection:
        while True:
            ret, frame = cap.read()
            if not ret:
                break

            frame = cv2.flip(frame, 1)  # Mirror the image

            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = face_detection.process(frame_rgb)

            if results.detections:
                for detection in results.detections:
                    bboxC = detection.location_data.relative_bounding_box
                    ih, iw, _ = frame.shape
                    bbox = int(bboxC.xmin * iw), int(bboxC.ymin * ih), \
                           int(bboxC.width * iw), int(bboxC.height * ih)
                    x, y, w, h = bbox  # Extract the x, y, width, and height of the bounding box

                    # Kirim koordinat tengah wajah ke Arduino
                    string = 'X{0:d}Y{1:d}'.format(x + w // 2, y + h // 2)
                    print(string)
                    ser.write(string.encode('utf-8'))
                    time.sleep(0.2)

                    # Gambar kotak pembatas wajah
                    cv2.rectangle(frame, bbox, (0, 0, 255), 3)
                    # Plot titik tengah wajah
                    cv2.circle(frame, (x + w // 2, y + h // 2), 2, (0, 255, 0), 2)

            # Gambar area persegi di tengah layar
            cv2.rectangle(frame, (640 // 2 - 30, 480 // 2 - 30), (640 // 2 + 30, 480 // 2 + 30), (255, 255, 255), 3)

            cv2.imshow('Face Detection', frame)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break


# Jalankan thread untuk deteksi tangan
hand_thread = threading.Thread(target=hand_detection)
hand_thread.start()
# Jalankan deteksi wajah
face_detection_worker()

# Pastikan untuk menunggu sampai thread selesai sebelum keluar
hand_thread.join()
# Hentikan video capture dan tutup jendela
cap.release()
cv2.destroyAllWindows()
ser.close()

pygame 2.3.0 (SDL 2.24.2, Python 3.8.2)
Hello from the pygame community. https://www.pygame.org/contribute.html
X159Y117
X152Y127
X153Y128
X161Y126
X182Y126
X208Y127
X241Y132
X282Y133
X334Y120
X365Y109


C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(


X359Y122
X358Y156
X357Y167
X332Y164
X316Y179
X327Y198
X337Y212
X355Y231
X377Y229
X378Y243
X357Y246
X348Y257
X330Y265
X322Y272
X295Y264
X280Y269
X269Y296
X259Y294
X307Y293
X342Y280
X362Y285
X378Y275
X375Y244
X344Y247
X333Y242
X335Y250
X329Y250
X331Y246
X325Y240
X320Y237
X358Y236
X410Y237
X419Y229
X422Y227
X407Y239
X384Y227
X373Y233
X367Y239
X354Y243
X337Y239


C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(


X338Y243
X328Y242
X329Y245
X328Y246
X320Y245
X317Y254
X313Y251
X316Y251
X326Y244
X322Y242
X323Y246
X324Y244
X319Y247
X318Y249
X320Y246
X319Y247
X317Y241
X324Y242
X320Y240
X315Y236
X291Y246
X257Y248
X245Y259


...

X332Y236


C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(


X328Y241


C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\riziq\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(


X317Y236
X296Y234
X301Y242
X308Y266
X341Y291
X370Y316
X362Y310
X320Y297
X266Y258
X256Y230
X304Y225
X402Y208
X442Y217
X431Y218
X389Y225
X357Y231
X303Y224
X298Y226
X326Y238
X342Y246
X355Y258
X352Y275
X369Y278
X332Y278
X290Y272
X270Y239
X264Y221
X242Y214
X232Y200
X265Y203
X289Y219
X337Y239
X416Y238
X419Y242
X428Y243
X410Y247
X27Y297
X32Y306
X368Y247
X389Y240
X373Y238
X358Y232
X308Y214
X279Y211
X255Y204
X247Y205
X258Y210
X309Y235
X334Y243
X353Y240
X364Y252
X395Y256
X367Y259
X335Y259
X265Y249
X190Y231
X171Y218
X189Y214
X230Y235
X258Y245
X276Y245
X278Y236
X270Y201
X273Y172
X288Y165
X311Y165
X315Y169
X315Y173
X312Y173
X310Y174
X305Y172
